# LLM Agents for Clinical Trial Management

In this notebook, we develop an agent-based Large Language Model (LLM) application to automate the evaluation of patients for potential clinical trials. By leveraging documents related to patients' medical histories, clinical policies, and trial inclusion criteria, such an agentic system can significantly enhance the quality of evaluations and reduce the time required.

We use limited demo databases and simple agentic tools and pipelines to demonstrate the capabilities of such LLM systems. For real-world applications, more comprehensive medical databases, in collaboration with clinical specialists, will be necessary. Additionally, more advanced tools (e.g., advanced Retrieval-Augmented Generation (RAG) methods) must be incorporated for real applications.

**Objective:** Automate clinical trial data management, analysis, compliance checks, and reporting.

## Key Components of This Design: 
1. Patient Data Collection: Gather comprehensive medical histories and relevant patient information.
2. Patient Data Analysis: Analyze patient data to identify key health indicators and relevant medical conditions.
3. Clinical Compliance Verification: Ensure patient data complies with clinical policies and trial eligibility criteria.
4. Trial Matching: Match patients with suitable clinical trials based on their medical profiles.
5. Human-in-the-Loop Interventions: Allow clinical experts to review and modify intermediate and final outcomes.

## Technologies and Tools Used:
- Local Models: Embedding of policies and trial information.
- Local Vector Stores: Information retrieval using metadata.
- Information Retrieval: Using metadata to find relevant documents.
- Chains of LLM Calls: Sequential LLM calls to perform complex tasks.
- Python Shell: LLM integration for performing numerical calculations.
- Agentic Graph: Deploying a workflow for evaluating patients and trials.
- Human-in-the-Loop: Clinical experts can intervene and modify the evaluation process and results.

In [ ]:
%load_ext autoreload
%autoreload 2
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")

# Loading the requied environment variables
_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")

# Optional, add tracing in LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Clinical Trial Management"

# Parameters

In [ ]:
from langchain_openai import ChatOpenAI


policy_vstore_created = False
trials_vstore_created = False
trials_db_downloaded = False
chromadb_path = "../../chroma_db"
policy_collection = "policy-chroma"
trial_collection = "trial-chroma"

modelID = "gpt-3.5-turbo"
n_retrieved_policies = 1 # Number of most relevant policies to retrieve per patient
# n_retries = 6
n_retrieved_trials = 6



model = ChatOpenAI(temperature = 0.0, model=modelID)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pprint
# A function for printing nicely
def nprint(text, indent=2):
    pp = pprint.PrettyPrinter(indent=indent)
    pp.pprint(text)

# Creating the demo datasets

## Patients database

We create a sample dataset of patients with their medical history, previous trials, and other relevant information.   
We use random names, diseases, and past or ongoing trials.  
This is a demo dataset limited information. However, for real cases, a more complit medical history of patients will be used.

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import json
import random

# Generate the sample data again for display
columns = ["patient_id", "name", "age", "medical_history", "previous_trials", "trial_status", "trial_completion_date"]
data = []

# Given names and surnames
names = ["John", "Jane", "Alice", "Michael", "Emily", "Daniel", "Sophia", "James", "Emma", "Oliver"]
surnames = ["Doe", "Smith", "Johnson", "Brown", "Davis", "Garcia", "Martinez", "Anderson", "Thomas", "Wilson"]

# Generate all possible unique combinations of names and surnames
combinations = [(name, surname) for name in names for surname in surnames]

# Shuffle the combinations to ensure randomness
random.shuffle(combinations)

# Select the first 100 unique combinations
unique_names = combinations[:100]

# Generate the full names
full_names = [f"{name} {surname}" for name, surname in unique_names]

with open('../../source_data/diseases_list.json', 'r') as file:
        trial_diseases =  json.load(file)

list_trial_diseases = list({disease for diseases in trial_diseases.values() for disease in diseases})

other_medical_conditions = ["Hypertension", "Diabetes", "Asthma", "Heart Disease", "Arthritis",
                      "Chronic Pain", "Anxiety", "Depression", "Obesity"]

all_conditions = list(set(list_trial_diseases + other_medical_conditions))

trial_statuses = ["Completed", "Ongoing", "Withdrawn"]

def random_date(start, end):
    return start + timedelta(days=random.randint(0, int((end - start).days)))

# start_date must be 2 years before now
start_date = datetime.now() - timedelta(days=365 * 2)
# start_date = datetime(2020, 1, 1)

# end_date must be a month before now
end_date = datetime.now() - timedelta(days=10)
# end_date = datetime(2023, 1, 1)

for i in range(1, 101):
    name = random.choice(full_names)
    age = random.randint(20, 80)
    medical_history = random.choice(all_conditions)
    
    if random.choice([True, False]):
        previous_trials = f"NCT0{random.randint(1000000, 9999999)}"
        trial_status = random.choice(trial_statuses)
        trial_completion_date = random_date(start_date, end_date).strftime('%Y-%m-%d')
    else:
        previous_trials = ""
        trial_status = ""
        trial_completion_date = ""
    if trial_status == "Ongoing":
        trial_completion_date = ""

    data.append((i, name, age, medical_history, previous_trials, trial_status, trial_completion_date))

df = pd.DataFrame(data, columns=columns)
# save df to csv
df.to_csv("patients.csv", index=False)
df.head(20)

Creating a local SQLite database for patients

In [ ]:
import os
import sqlite3
import pandas as pd

overwrite = True

database_file = 'patients_database.db'
df = pd.read_csv('patients.csv')
if overwrite or not os.path.exists(database_file):
    if os.path.exists(database_file):
        os.remove(database_file)
    conn = sqlite3.connect(database_file)
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS patients (
        patient_id INTEGER PRIMARY KEY,
        name TEXT,
        age INTEGER,
        medical_history TEXT,
        previous_trials TEXT,
        trial_status TEXT,
        trial_completion_date TEXT
    )
    ''')

    # Insert DataFrame into SQLite table
    df.to_sql('patients', conn, if_exists='append', index=False)

    # Commit and close the connection
    conn.commit()
else:
    conn = sqlite3.connect(database_file)
    cursor = conn.cursor()

query = 'SELECT * FROM patients'

# Execute the query and fetch all results
cursor.execute(query)
rows = cursor.fetchall()

# Optionally, you can get the column names
column_names = [description[0] for description in cursor.description]

# Convert the results to a Pandas DataFrame for better readability
df = pd.DataFrame(rows, columns=column_names)
print(len(df))

# Display the DataFrame
conn.close()

db = database_file
df.head(10)

## Clinical Trials database

We download a sample clinical trials database from a [publicly available source]('https://raw.githubusercontent.com/futianfan/clinical-trial-outcome-prediction/main/data/raw_data.csv').   
Details are provided in the [helper function](../../src/helper_functions.py)

In [ ]:
import sys
module_dir = os.path.abspath('../../src')  # Gets the absolute path to the src directory
sys.path.append(module_dir)
from helper_functions import dataset_create_trials

overwrite = False
if trials_db_downloaded and not overwrite:
    print(f'Trials database already downloaded and stored in {trials_path}')
    df_trials = pd.read_csv(trials_path)
else:
    df_trials, trials_path = dataset_create_trials(status='recruiting')
    trials_db_downloaded = True

# print the ctriteria column value froom 4 random rows of the dataferam. use a for loop
df_samples = df_trials.sample(4).reset_index(drop=True)
for i in range(4):
    print(f'---- Sample {i+1} ----')
    print(df_samples.iloc[i]['criteria'])


# Creating Vectore Stores

## Vstore for policy database

To create a vector store for the policy database, we will use perform local embedding via NomicEmbeddings.   
Also, to treat each policy separately, we split the policies main document into per policy chunks.   
Note that this is a problem specific strategy, and may not be appropriate for all RAG applications.

In [ ]:
import re

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_nomic.embeddings import NomicEmbeddings
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document
import chromadb

# vstore_delete = False
vstore_load = True
markdown_path = "../../source_data/instut_trials_policy.md"

with open(markdown_path, "r") as f:
    policy_text = f.read()


doc_splits = [
    Document(page_content= txt, metadata={"source": markdown_path}) for txt in re.split(r"(?=\n###)", policy_text)
    ]

# the title of markdown is not required in the split
doc_splits = doc_splits[1:]    

print(len(doc_splits))
doc_splits

persistent_client = chromadb.PersistentClient(path = chromadb_path)

if vstore_load == False:
    vectorstore.delete_collection()


# if chromadb_path exists
if os.path.exists(chromadb_path) and vstore_load:
    vectorstore = Chroma(
        client=persistent_client,
        collection_name=policy_collection,
        embedding_function=NomicEmbeddings(model="nomic-embed-text-v1.5",
                                inference_mode='local'),
    )
    print('Vectore store loaded from existing chromadb')
    # vectorstore._collection.count()
else:
    vectorstore = Chroma.from_documents(
        documents=doc_splits,
        client=persistent_client,
        collection_name=policy_collection,
        # persist_directory=chromadb_path,
        embedding=NomicEmbeddings(model="nomic-embed-text-v1.5",
                                inference_mode='local'),
    )
    print('Vectore store created and stored in persistent chromadb client')
print(vectorstore._collection.count())
doc_splits


## Vstore for trials

For simplicy, we just work on 3 main categories of cancer, lukemia, and mental health problems.   
This selection gives us a managable set of trials as a demo application.   
However, for a more proper application, categories can be extended with the help of domain specialists.

The vectore store is created using chromadb and as a persistent client.

In [ ]:
from helper_functions import disease_map

vstore_delete = False

trial_docs = []
for i, row in df_trials.iterrows():
    disease = disease_map(row['diseases'])
    if disease == 'other_conditions':
        continue
    doc = Document(
        page_content=row['criteria'],
        metadata={
            "nctid": row['nctid'],
            "status": row['status'],
            # "why_stop": row['why_stop'],
            # "label": row['label'],
            # "phase": row['phase'],
            "diseases": str(row['diseases']),
            "disease_category": disease[0],
            "drugs": row['drugs'],            
        }
    )
    trial_docs.append(doc)
# trial_docs

print(trial_docs[0].metadata)

list_remove = set()
for i, doc in enumerate(trial_docs):
    if len(doc.page_content)>10000:
        list_remove.add(i)
        # print(doc.metadata)
    if doc.metadata['disease_category'] == 'other_conditions':
        list_remove.add(i)
        # print(doc.metadata)
# remove list_remove indexes from trial_docs
trial_docs = [doc for i, doc in enumerate(trial_docs) if i not in list_remove]

# print(len(trial_docs[11].page_content))
# trial_docs.pop(11)

persistent_client = chromadb.PersistentClient(path = chromadb_path)

if vstore_delete == True:
    vectorstore_trials.delete_collection()
    trials_vstore_created = False
    vstore_delete = False
    print("vstore deleted")

vectorstore_trials = Chroma(
    client=persistent_client,
    collection_name=trial_collection,
    embedding_function=NomicEmbeddings(model="nomic-embed-text-v1.5",
                            inference_mode='local'),
)
if vectorstore_trials._collection.count() == 0:
    vectorstore_trials = Chroma.from_documents(
        documents=trial_docs,
        client=persistent_client,
        collection_name=trial_collection,
        # persist_directory=chromadb_path,
        embedding=NomicEmbeddings(model="nomic-embed-text-v1.5",
                                inference_mode='local'),
    )
    trials_vstore_created = True
    print("Vectorstore is created now")
else:
    print("Loading the vectorstore from persistent client")

print(vectorstore_trials._collection.count())
trial_docs


# Tools:
Here we define some tools that the agent needs to use for evaluation of the patient, general policies, and clinic trials.

### Patient Data Extractor

In [ ]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

def get_patient_data(patient_id: int) -> dict:
    """Fetch all fields for the patient based on the given patient_id as an integer.

    Returns:
        A dictionary containing the patient's medical history.        
    """    

    conn = sqlite3.connect(db)
    cursor = conn.cursor()    
    query = 'SELECT * FROM patients WHERE patient_id=?'
    cursor.execute(query, (patient_id,))
    patient_data = cursor.fetchone()
    # rows = cursor.fetchall()
    column_names = [column[0] for column in cursor.description]
    conn.close()
    if patient_data is None:
        return None
    else:    
        results = dict(zip(column_names, patient_data))    
    return results

# Also possible to add a new patient
def add_patient_data(patient_data: dict):    
    """Adds a new patient to the SQLite database."""
    
    name = patient_data['name']
    age = patient_data['age']
    medical_history = patient_data['medical_history']
    previous_trials = patient_data['previous_trials']
    trial_status = patient_data['trial_status']
    last_trial_dates = patient_data['last_trial_dates']

    conn = sqlite3.connect(db)
    cursor = conn.cursor()
    
    # Insert the new patient data into the database
    cursor.execute('''
    INSERT INTO patients (name, age, medical_history, previous_trials, trial_status, last_trial_dates)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (name, age, medical_history, previous_trials, trial_status, last_trial_dates))
    
    conn.commit()
    conn.close()

In [ ]:
sample_patient_id = 56
patient_data = get_patient_data(sample_patient_id)
patient_data

### Profile Maker

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

parser = StrOutputParser()
prompt_profile = PromptTemplate(
    template="""
    You are the Clinical Research Coordinator in the screening phase of a clinical trial. 
    Use the following patient data to write the patient profile for the screening phase.
    The patient profile is a summary of the patient's information in continuous text form.    
    If they had no previous trial participation, exclude trial status and trial completion date.\n
    Do not ignore any available information.\n 
    In a separate section of the profile, also suggest medical trial categories related to patient's disease history.\n    
    Write the patient profile in 3 to 4 short sentences.\n\n
    {patient_data}""",
    input_variables=["patient_data"],
)
# model = ChatOpenAI(temperature = 0.0, model=modelID)

chain_profile = prompt_profile | model | parser
# to preserve the patient's private information

if patient_data.get('name'):
    del patient_data['patient_id']
    del patient_data['name']

patient_profile = chain_profile.invoke({'patient_data': patient_data})
nprint(patient_profile) # patient_profile

In [ ]:
# to preserve the patient's private information
if patient_data.get('name'):
    del patient_data['patient_id']
    del patient_data['name']

patient_profile = chain_profile.invoke({'patient_data': patient_data})
nprint(patient_profile) # patient_profile

### Policy Retriever


In [ ]:
retriever_policy = vectorstore.as_retriever(search_kwargs={"k": n_retrieved_policies})

# sample policy retrieval
question = "\ntrial_completion_date: 2024-04-01"
docs_retrieved = retriever_policy.get_relevant_documents(question)
print(len(docs_retrieved))
doc_txt = docs_retrieved[0].page_content
docs_retrieved

### Policy Evaluator with Python REPL

In [ ]:
from langchain_experimental.utilities import PythonREPL
from typing import Annotated, List
# from langchain_experimental.tools import PythonREPLTool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_openai_tools_agent
# import functools
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import Tool


def policy_repl(policy, patient_profile):

    class codesc(BaseModel):
        code: str = Field(
            description="The code to execute. IMPORTANT: the last line of code must always finish with a print(...)."
        )

    python_repl = PythonREPL()
    python_repl_tool = Tool(
        name="python_repl",
        description="""
        A Python shell. Use this to execute python commands. Input should be a valid python command. IMPORTANT: alwyas print the output value out with `print(...)`.

        Wrong code example:
        ```python
        A = 12
        B = 20
        A_bigger = A > B    
        ```

        Correct code example:
        ```python
        A = 12
        B = 20
        A_bigger = A > B
        print("A_bigger")
        ```
        """,
        func=python_repl.run,
        args_schema=codesc,
    )        

    prompt_repl = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a Principal Investigator (PI) for evaluating patients for clinical trials." 
                "You are asked to compare the patient profile document to the institution policies document."
                "You must determine if the patient is eligible based on the following documnents or not."
                # "Use the following institution policy(s) to find if the patient is eligibile for trials."
                "\n #### Here is the patient profile document: \n {patient_profile}\n\n"
                # "\n Here is/are the retrieved policy(s) document: \n {context}\n\n"

                "-------------------\n"
                
                "Today date is {date}.\n\n"

                # "EXCLUSION terms in the patient profile should be removed from policies.\n"                                                
                "Do not use your own knowledege regarding clinical trials. Restrict your judgment Only to the information given in documents.\n"
                "You have access to the following tools: {tool_names}."
                # "\n You may generate only safe python code."
                "\n IMPORTANT: Do not use this tool {tool_names} when thre is no numerical or date related information in the policy document.\n\n"
                
                "Give a binary 'yes' or 'no' score in the response to indicate whether the patient is eligible according to the given policy ONLY."
                "If the patient is not eligible then also include the reason in your response.", 
            ),
            # MessagesPlaceholder(variable_name="examples"),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),            
        ]
    )   

    tools = [python_repl_tool]
    # patient_profile = state["patient_profile"]

    date = datetime.today().date()
    prompt_repl = prompt_repl.partial(tool_names=", ".join([tool.name for tool in tools]))
    prompt_repl = prompt_repl.partial(date=date)    
    prompt_repl = prompt_repl.partial(patient_profile=patient_profile)    

    agent = create_openai_tools_agent(model, tools, prompt_repl)
    agent_with_tool = AgentExecutor(agent=agent, tools=tools)

    message = f"Here is the retrieved policy document: \n {policy}\n\n"
    # nprint(message)    
    result = agent_with_tool.invoke({"messages": [HumanMessage(content=message)]} )
    return result

In [ ]:
# sample policy evaluation
policy = docs_retrieved[0].page_content
# nprint(policy)
policy_repl(policy, patient_profile)

### Trials Retriever 

We use SelfQueryRetriever module that also uses metadata information. This retrival can quickly find relevant trials based on metadata such as disease category, drug names, etc.   
The value of such retrival becomes important for real applications, where patient profiles contain more comphrehensive medical histories.

In [ ]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name="disease_category",
        description="Defines the disease group of patients related to this trial. One of ['cancer', 'leukemia', 'mental_health']",
        # description="The trial is for patients when their disease is related to this category. One of ['cancer', 'leukemia', 'mental_health']",
        type="string",
    ),
    AttributeInfo(
        name="drugs",
        description="List of drug names used in the trial",
        type="str",
    ),    
]

document_content_description = "The list of patient conditions to include or exclude them from the trial"
retriever_trial_sq = SelfQueryRetriever.from_llm(
    model,
    vectorstore_trials,
    # vectorstore_trials_mpnet,
    document_content_description,
    metadata_field_info
    # enable_limit=True
)

In [ ]:
# sample trial retrieval
question = f"""
Which trials are relevant to the patient with the following medical history?\n
patient_profile: {patient_profile}
"""        
docs_retrieved = retriever_trial_sq.get_relevant_documents(question)
print(docs_retrieved[0].metadata)
print(docs_retrieved[0].page_content)

### Trial Grader

In [ ]:
class grade(BaseModel):
        """The result of the trial's relevance check as relevance score and explanation."""

        relevance_score: str        
        explanation: str = Field(description="Reasons to the given relevance score.")        
        further_information: str

prompt_grader = PromptTemplate(
        template=""" 
        You are a Principal Investigator (PI) for evaluating patients for clinical trials.\n
        Your task is to evaluate the relevance of a clinical trial to the given patient's medical profile. \n
        
        
        The clinical trial is related to these diseases: {trial_diseases} \n
        Here are the inclusion and exclusion criteria of the trial: \n\n {document} \n\n
        
        ===============                
        Use the following steps to determine relevance and provide the necessary fields in your response: \n
        1- If the patient's profile meets any exclusion criteria, then the trial is not relevant --> relevance_score = 'No'. \n
        2- If the patient has or had the trial's inclusion diseases, then it is relevant --> relevance_score = 'Yes'.\n        
        3- If the patient did not have the trial's inclusion diseases, then it is not relevant --> relevance_score = 'No'.\n
                       
        Example 1: 
The patient has Arthritis and the trial is related to pancreatic cancer. --> relevance_score = 'No' \n
        
        Example 2: 
The patient has pancreatic cancer and the trial is also related to carcinoma pancreatic cancer. --> relevance_score = 'Yes' \n

        Example 3: 
The patient has pancreatic cancer and the trial is related to breast cancer or ovarian cancer. --> relevance_score = 'No'. \n 

        Bring your justification in the explanation. \n

        Mention further information that is needed from the patient's medical history related to the trial's criteria \n

        ===============
        Here is the patient's medical profile: {patient_profile} \n\n
        """,
        input_variables=["document", "patient_profile", "trial_diseases"],
    )

llm_with_tool = model.with_structured_output(grade)
retrieval_grader = prompt_grader | llm_with_tool

In [ ]:
doc = docs_retrieved[0]
doc_txt = doc.page_content
trial_diseases = doc.metadata.get("diseases")
trial_score = retrieval_grader.invoke({"patient_profile": patient_profile, "document": doc_txt, "trial_diseases": trial_diseases})

for score in trial_score:
    nprint(score)

### Hallucination Grader


In [ ]:
class GradeHallucinations(BaseModel):
    """Binary score and explanation for whether the LLM's generated answer is grounded in / supported by the facts in the patient's medical profile."""
    # """Binary score for hallucination present in generation answer."""


    binary_score: str = Field(
        description="Answer is grounded in the patient's medical profile, 'yes' or 'no'"
    )
    Reason: str = Field(description="Reasons to the given relevance score.")


# LLM with function call
llm_with_tool = model.with_structured_output(GradeHallucinations)

# Prompt
system = """You are a grader assessing whether an LLM generation is grounded in / supported by the facts in the patient's medical profile. \n 
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the facts in the patient's medical profile."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Patient's medical profile: \n\n {patient_profile} \n\n LLM generation: {explanation}"),
    ]
)

hallucination_grader = hallucination_prompt | llm_with_tool
print(trial_score.relevance_score)
explanation = trial_score.explanation
nprint(explanation)
# docs = patient_profile
score = hallucination_grader.invoke({"patient_profile": patient_profile, "explanation": explanation})
grade = score.binary_score
print(grade)
nprint(score.Reason)

### Profile Rewriter

In [ ]:
system = """
A trial cross match resulted in no trials for the patient.
As a clinical specialist write a medical profile for this patient and see if their disease(s) can be relevant to any of these categories of mental_health, cancer, or leukemia.
If yes, then suggest relevant medical trial categories for the agent.
If no, then do not add anything there.

Your output must be as below:
<a text summart of original profile>
Suggested relevant trials:
<bullet points of relevant medical trial categoriyes from the above with a one line reason>

Only include categories which can be related to patient diseases in more often cases.
Disregard categories which ocasionaly or in some cases can be relevant to patient diseases.

example:
The patient is a X-year-old with a medical history of Y. They have participated ........  previous trials, and their trial status and completion date .........
Suggested relevant trials:
category X: [patient's disease] can be related to X due to Y.
"""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is a patient data:\n\n {patient_data} \n write a patient profile.",
        ),
    ]
)

profile_rewriter_chain = re_write_prompt | model | StrOutputParser()
profile_rewriter_chain.invoke({"patient_data": patient_data})

# Graph

## State

In [ ]:
from typing import Annotated, List
from typing_extensions import TypedDict
from langgraph.graph.message import AnyMessage, add_messages
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_core.documents import Document

class AgentState(TypedDict):
    # messages: Annotated[Sequence[BaseMessage], operator.add]
    last_node: str
    patient_prompt: str
    patient_id: int
    policy_eligible: bool
    rejection_reason: str
    patient_data: dict
    patient_profile: str
    revision_number: int
    max_revisions: int
    trial_searches: int
    max_trial_searches: int    
    policies: List[Document]
    checked_policy: Document
    unchecked_policies: List[Document]
    trials: List[Document]
    relevant_trials: list[dict]
    ask_expert: str

## Agent

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langchain_core.pydantic_v1 import BaseModel, Field
from operator import itemgetter
from typing import Literal
from langgraph.graph import StateGraph, END


model = ChatOpenAI(model="gpt-3.5-turbo")  #reduce inference cost

memory = SqliteSaver.from_conn_string(":memory:")

class Patient_ID(BaseModel):
    patient_id: int
    # no_id: bool

def patient_collector_node(state: AgentState):
    patient_data_prompt = """You are a helpful assistance in extrcting patient's medical history.
Based on the following request identify and return the patient's ID number.
"""

    response = model.with_structured_output(Patient_ID).invoke([
        SystemMessage(content=patient_data_prompt),
        HumanMessage(content=state['patient_prompt'])
    ])
    patient_id = response.patient_id
    # print(response)
    print(f"Patient ID: {patient_id}")
    patient_data = get_patient_data(patient_id)
    print(patient_data)
    if patient_data is not None:        
        if patient_data.get('name'):
            del patient_data['patient_id']
            del patient_data['name']
        patient_profile = chain_profile.invoke({'patient_data': patient_data})

    return {
        "last_node": "patient_collector",
        "patient_data": patient_data,
        "patient_profile": patient_profile,
        "patient_id": patient_id,
        "revision_number": state.get("revision_number", 1) + 1,
        'policy_eligible': 'N/A'
        }

def policy_search(state: AgentState) -> List[Document]:    
    question = f"""
    Does the following patient profile comply with the institutional policies?\n
    {patient_profile}
    """
    docs_retrieved = retriever_policy.get_relevant_documents(question)
    if docs_retrieved is None:
        docs_retrieved = retriever_policy.get_relevant_documents(question)
    return {
        "last_node": "policy_search",
        "policies": docs_retrieved,
        "unchecked_policies": docs_retrieved.copy(),
    }

def policy_evaluator(state: AgentState, policy_doc: Document = None):
    if policy_doc is None:
        policy_doc = state['unchecked_policies'][0]
    
    policy_header = policy_doc.page_content.split('\n', 2)[1]
    print(f'Evaluating Policy:\n {policy_header}')
    
    policy = policy_doc.page_content
    
    result = policy_repl(policy, state['patient_profile'])
    # nprint(result['output'])

    class eligibility(BaseModel):
        """ Give the patient's eligibility result."""

        eligibility: str = Field(description="Patient's eligibility for the clinical trial. 'yes' or 'no'")
        reason: str = Field(description="The reason(s) only if the patient is not eligible for clinical trials. Othereise use N/A")

        class Config:
            schema_extra = {
                "example": {
                    "eligibility": 'yes',
                    "reason": "N/A",
                },
                "example 2": {
                    "eligibility": 'no',
                    "reason": "The patient is pregnant at the moment.",
                },                
            }

    llm_with_tools = model.bind_tools([eligibility])
    message = f"""Evaluation of the patient's eligibility:
    {result['output']}\n\n
    Is the patient eligible according to this policy?"""
    response = llm_with_tools.invoke(message)
            
    state["policy_eligible"] = response.tool_calls[0]['args']['eligibility'] == 'yes'
    state["rejection_reason"] = response.tool_calls[0]['args']['reason']
    
    # remove the first element of unchecked_policies
    state['unchecked_policies'].pop(0)
    state["revision_number"] = state.get("revision_number", 1) + 1
    state['checked_policy'] = policy_doc
    state['last_node'] = 'policy_evaluator'

    return state

def trial_search(state: AgentState):
    """
    This node searches the trial database to retrieve a list of clinical trials that match the patient's medical history.
    """

    patient_profile = state['patient_profile']

    question = f"""
    Which trials are relevant to the patient with the following medical history?\n
    patient_profile: {patient_profile}
    """        
    docs_retrieved = retriever_trial_sq.get_relevant_documents(question)
    # print(state)
    trial_searches = state.get('trial_searches') or 0
    return {
        'last_node': 'trial_search',
        'trials': docs_retrieved,
        'trial_searches': trial_searches + 1,
        }

def grade_trials(state: AgentState, trials: List[Document] = None):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("----- CHECKING THE TRIALS RELEVANCE TO PATIENT PROFILE ----- ")
    
    if trials is None:
        trials = state['trials']
    patient_profile = state['patient_profile']

    # Score each doc
    relevant_trials = []
    for trial in trials:
        doc_txt = trial.page_content
        trial_diseases = trial.metadata['diseases']
        nctid = trial.metadata['nctid']
        print(f"---GRADER: TRIAL {nctid}: ---") 
        trial_score = retrieval_grader.invoke(
            {
                "patient_profile": patient_profile, 
                "document": doc_txt, 
                "trial_diseases": trial_diseases
            }
        )
            
        relevance_score = trial_score.relevance_score
        if relevance_score.lower() == "yes":   
            # Hullucination check         
            explanation = trial_score.explanation            
            factual_score = hallucination_grader.invoke({"patient_profile": patient_profile, "explanation": explanation})
            factual_score_grade = factual_score.binary_score            
            if factual_score_grade == "no":
                print("--- HALLUCINATION: MODEL'S EXPLANATION IS NOT GROUNDED IN PATIENT PROFILE --> REJECTED---")

        if relevance_score.lower() == "yes" and factual_score_grade == "yes":
            print(f"---TRIAL RELEVANT---")                        
        else:
            print(f"--- TRIAL NOT RELEVANT---")

        trial_score_dic = dict(trial_score)
        trial_score_dic['nctid'] = nctid            
        relevant_trials.append(trial_score_dic)    
        
    return {
        'last_node': 'grade_trials',
        "relevant_trials": relevant_trials
        }

def profile_rewriter(state: AgentState):
    patient_data = state['patient_data']
    patient_profile_rewriten = profile_rewriter_chain.invoke({"patient_data": patient_data})
    # print in capitals
    print("--- PROFILE REWRITER: PATIENT'S PROFILE REWRITEN ---")
    # state['patient_profile'] = patient_profile_rewriten
    return {
        'last_node': 'profile_rewriter',
        'patient_profile': patient_profile_rewriten
    }
    

# ========= continue functions =========
def should_continue_patient(state: AgentState):
    # end = False
    if state.get("patient_data"):
        # print('patient data found')
        print(state.get("patient_profile", 'No profile generated'))
        return "policy_search"
    else:
        print('Patient data not found in the database')
        return END
        
    if state["revision_number"] > state["max_revisions"]:
        print('max revision reached')
        # print(state.get("patient_data", 1))
        return END        

def should_continue_policy(state: AgentState):
    if state["revision_number"] > state["max_revisions"]:
        print('max revision reached')        
        return END
    more_policies = len(state["unchecked_policies"]) > 0
    if state["policy_eligible"]:
        if more_policies:
            print("-------------- Check the next policy --------------")
            return "policy_evaluator"
        else:
            print("-------------- Patient passed the general trials policies --------------")
            return "trial_search"
    else:
        print("Patient did not pass this policy.\n")
        print(state['checked_policy'].page_content)
        # print("Patient did not pass the general trials policies.\n")        
        print('Rejection Reason: ')
        nprint(state['rejection_reason'])
        print("You can correct the patient's medical profile if required.")        
        return END

def should_continue_trials(state: AgentState):
    relevant_trials = state["relevant_trials"]    
    has_relevant_trial = any(trial['relevance_score'] == 'Yes' for trial in state["relevant_trials"])

    if state["trial_searches"] > state["max_trial_searches"]:
        print('max trial searches reached')
        next = END    
    # elif not state["relevant_trials"]:
    elif has_relevant_trial == False:
        print('----- No relevant trials found --> rewriting the patient profile -----')
        next = "profile_rewriter"        
    else:
        next = END
        print('to end')
    return next


# ==== define graph nodes====
builder = StateGraph(AgentState)
builder.set_entry_point("patient_collector")
builder.add_node("patient_collector", patient_collector_node)
builder.add_node("policy_search", policy_search)
builder.add_node("policy_evaluator", policy_evaluator)
builder.add_node("trial_search", trial_search)
builder.add_node("grade_trials", grade_trials)
builder.add_node("profile_rewriter", profile_rewriter)

# ===== define graph edges====
builder.add_conditional_edges(
    "patient_collector", 
    should_continue_patient, 
    {END: END    
    ,"policy_search": "policy_search"
    }
)

builder.add_conditional_edges(
    "policy_evaluator", 
    should_continue_policy, 
    {
    "trial_search": "trial_search"
    , "policy_evaluator": "policy_evaluator"
    , END: END
    }
)

builder.add_edge("policy_search", "policy_evaluator")
builder.add_edge("trial_search", "grade_trials")
builder.add_edge("profile_rewriter","trial_search")
# builder.set_finish_point("grade_trials")

builder.add_conditional_edges(
    "grade_trials", 
    should_continue_trials, 
    {
    "profile_rewriter": "profile_rewriter"    
    , END: END
    }
)



graph = builder.compile(
    checkpointer=memory
)

from IPython.display import Image

Image(graph.get_graph().draw_png())

Running with no gui and intrupts:

In [ ]:
import langchain
langchain.debug = False
sample_patient_id = 2
thread = {"configurable": {"thread_id": "0"}}
events_gen = graph.stream({
    'patient_prompt': f"Is patient {sample_patient_id} eligible for any medical trial?",    
    "max_revisions": 10,
    "revision_number": 1,
    'max_trial_searches':3,
}, thread)
for event in events_gen:
    print(event)
# event = events_gen.__next__()

# Application GUI

In [ ]:
from helper_gui import trials_gui

memory = SqliteSaver.from_conn_string(":memory:")
graph = builder.compile(
    checkpointer=memory
    , interrupt_after=['patient_collector', 'policy_search', 'trial_search', 'grade_trials', 'profile_rewriter']
    # , interrupt_after=['grade_trials','profile_rewriter']
)

try:
    app.demo.close()
except:
    print('initial run')
app = trials_gui(graph)    
app.demo.launch(server_name="127.0.0.1", server_port=7959)

# Possible Further Improvements for Real Application:

To enhance the robustness of the pipeline and effectively manage diverse patient and trial conditions, it is essential to implement advanced methodologies, particularly when handling large databases of patients, trials, and policies.

Recommended Improvements:
1. Graph-Based Retrieval-Augmented Generation (RAG):
- Implement graph-based RAG to focus on the relationships between entities across different databases. For instance, map connections between patients, diseases, drugs, and trials to provide more accurate and relevant information retrieval.

2. Advanced RAG pipeline:
- The output of RAG pipelines can be improved by benefiting from more advanced techniques such as Adaptive RAG, Corrective RAG, and Self-RAG. Such pipelines are more robust againts hallucinations and wrong inferences in LLMs outputs.

4. Advanced Chain of Thought Processing:
- Employ a more sophisticated chain of thought process to cross-match the patient's profile with the detailed information provided in each trial or policy. This approach enhances the decision-making capability of the system by considering multiple factors simultaneously.

1. Fine-Tuning the Large Language Model:
- Fine-tune the large language model using a diverse set of medical trial data to improve the accuracy and relevance of its inferences. This customization ensures the model better understands the nuances and specifics of clinical trial data.

1. Cyclic Graphs for Iterative Evaluation:
- Utilize cyclic graphs to iteratively evaluate the patient's profile against various trials and policies. This iterative approach allows the system to refine its matches through multiple rounds of evaluation, ultimately identifying the best possible trial for the patient.